In [139]:
import numpy as np

In [140]:
from joblib import Parallel, delayed
from itertools import product

In [141]:
sampler_param = [{"name":"cGAN", "candidate_parameters":{"A":[1,2,3], "B":[1,2,3]}},
                 {"name":"SMOTE", "candidate_parameters":{"K":[1,2,5]} }
           ]
model_param = [{"name":"logreg", "candidate_parameters":{"C":[1,10,100]}}]

In [142]:
sampler_fun = {"cGAN":None,
                "SMOTE":None
}
model_fun = {"logreg":None
}

In [143]:
from sklearn.model_selection import ParameterGrid

def make_validation_grid(parameter_dict, object_dict):
    out = []
    for model in parameter_dict:
        if model["name"] not in object_dict.keys():
           raise KeyError("Model name not found in model objects")

        param_grid = list(ParameterGrid(model["candidate_parameters"]))

        for param in param_grid:
            out.append({"name":model["name"], 
                             "base":object_dict[model["name"]], 
                             "params":param})
    return out

In [150]:
sampler_grid = make_validation_grid(sampler_param, sampler_fun)

In [158]:
X = np.random.rand(1000,10)
y = np.random.randint(0,2,1000)

In [137]:
from sklearn.model_selection import StratifiedKFold
cv_sampler = StratifiedKFold(n_splits=5)
cv_model = StratifiedKFold(n_splits=3)

In [ ]:
def _fit_and_score_sampler(sampler, model_grid, X, y, train, test, **kwargs):
    # Oversample training data
    
    
    
    # Train and evaluate all models
    
    
    # Return results

In [ ]:
parallel = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
                            pre_dispatch=self.pre_dispatch)

fit_and_score_kwargs = dict(scorer=scorers,
                            fit_params=fit_params,
                            return_train_score=self.return_train_score,
                            return_n_test_samples=True,
                            return_times=True,
                            return_parameters=False,
                            error_score=self.error_score,
                            verbose=self.verbose)
results = {}
with parallel:
    all_candidate_params = []
    all_out = []

    def evaluate_candidates(candidate_params):
        candidate_params = list(candidate_params)
        n_candidates = len(candidate_params)

        if self.verbose > 0:
            print("Fitting {0} folds for each of {1} candidates,"
                  " totalling {2} fits".format(
                      n_splits, n_candidates, n_candidates * n_splits))

        out = parallel(delayed(_fit_and_score)(clone(base_estimator),
                                               X, y,
                                               train=train, test=test,
                                               parameters=parameters,
                                               **fit_and_score_kwargs)
                       for parameters, (train, test)
                       in product(candidate_params,
                                  cv.split(X, y, groups)))

        if len(out) < 1:
            raise ValueError('No fits were performed. '
                             'Was the CV iterator empty? '
                             'Were there no candidates?')
        elif len(out) != n_candidates * n_splits:
            raise ValueError('cv.split and cv.get_n_splits returned '
                             'inconsistent results. Expected {} '
                             'splits, got {}'
                             .format(n_splits,
                                     len(out) // n_candidates))

        all_candidate_params.extend(candidate_params)
        all_out.extend(out)

        nonlocal results
        results = self._format_results(
            all_candidate_params, scorers, n_splits, all_out)
        return results